In [1]:
import requests
import pandas as pd
import os
import datetime

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import and_
from sqlalchemy.types import Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

import pymysql
import warnings

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re


In [2]:
# Imported raw data
url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
url_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

In [3]:
# Convert the data to a dataFrame
df_confirmed = pd.read_csv(url_confirmed)
df_deaths = pd.read_csv(url_deaths)
df_recovered = pd.read_csv(url_recovered)


In [4]:
# Transpose columns to rows
df_confirmed_transposed = df_confirmed.transpose()
df_deaths_transposed = df_deaths.transpose()
df_recovered_transposed = df_recovered.transpose()


In [5]:
df_confirmed_transposed

,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,224
Province/State,Anhui,Beijing,Chongqing,Fujian,Gansu,Guangdong,Guangxi,Guizhou,Hainan,Hebei,...,"Jackson County, OR","Jefferson County, WA","Kershaw County, SC","Klamath County, OR","Madera County, CA","Pierce County, WA","Plymouth County, MA","Santa Cruz County, CA","Tulsa County, OK","Montgomery County, TX"
Country/Region,Mainland China,Mainland China,Mainland China,Mainland China,Mainland China,Mainland China,Mainland China,Mainland China,Mainland China,Mainland China,...,US,US,US,US,US,US,US,US,US,US
Lat,31.8257,40.1824,30.0572,26.0789,36.0611,23.3417,23.8298,26.8154,19.1959,38.0428,...,42.3345,47.7425,34.3672,42.6953,37.2519,47.0676,42.1615,36.9741,36.1593,30.3213
Long,117.226,116.414,107.874,117.987,103.834,113.424,108.788,106.875,109.745,114.515,...,-122.765,-123.304,-80.5883,-121.614,-119.696,-122.129,-70.7928,-122.031,-95.941,-95.4778
1/22/20,1,14,6,1,0,26,2,1,4,1,...,0,0,0,0,0,0,0,0,0,0
1/23/20,9,22,9,5,2,32,5,3,5,1,...,0,0,0,0,0,0,0,0,0,0
1/24/20,15,36,27,10,2,53,23,3,8,2,...,0,0,0,0,0,0,0,0,0,0
1/25/20,39,41,57,18,4,78,23,4,19,8,...,0,0,0,0,0,0,0,0,0,0
1/26/20,60,68,75,35,7,111,36,5,22,13,...,0,0,0,0,0,0,0,0,0,0
1/27/20,70,80,110,59,14,151,46,7,33,18,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Insert tables to Covid Database in MySQL with 3 tables

USER = "root"
PASSWORD = "password"
HOST = "127.0.0.1"
PORT = "3306"
DATABASE = "Covid"
TABLENAME_CONFIRMED = "time_series_confirmed"
TABLENAME_DEATHS = "time_series_deaths"
TABLENAME_RECOVERED = "time_series_recovered"

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}", echo=False)
                                       
engine.execute(f"USE {DATABASE}")

# Drop the tables so we can insert new updated data

engine.execute(f"DROP TABLE IF EXISTS {TABLENAME_CONFIRMED}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME_DEATHS}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME_RECOVERED}")

# Insert the dataframe into our database Covid
df_confirmed.to_sql(name="time_series_confirmed", con=engine, index=False)
df_deaths.to_sql(name="time_series_deaths", con=engine, index=False)
df_recovered.to_sql(name="time_series_recovered", con=engine, index=False)
